In [1]:
SEED_NUMBER = 10
import random
random.seed(SEED_NUMBER)

This cell to run any additional requirement that your code need <br>


In [2]:
!pip install memory-profiler >> log.txt
!pip install -r requirements.txt


In [3]:
import os

In [ ]:
# import os
# import numpy as np
# from sklearn.cluster import MiniBatchKMeans
# import logging

# class IVF_PQ:
#     def __init__(self, n_subvectors, n_bits, n_clusters, dimension=64, folder_path='index', random_state=42):
#         self.n_subvectors = n_subvectors
#         self.n_bits = n_bits
#         self.n_clusters_per_subvector = 2 ** n_bits
#         self.n_clusters = n_clusters
#         self.dimension = dimension
#         self.random_state = random_state
#         self.folder_path = folder_path

#         if dimension % n_subvectors != 0:
#             raise ValueError("Dimension must be divisible by n_subvectors")

#     def build(self, vectors):
#         n_vectors = vectors.shape[0]
#         logging.info("Training IVF...")
#         kmeans = MiniBatchKMeans(n_clusters=self.n_clusters, random_state=self.random_state, batch_size=10000)
#         labels = kmeans.fit_predict(vectors)
#         cluster_centers = kmeans.cluster_centers_.astype(np.float32)

#         sort_idx = np.argsort(labels)
#         vectors_sorted = vectors[sort_idx]
#         original_ids = np.arange(n_vectors)[sort_idx]

#         offsets = np.zeros(self.n_clusters + 1, dtype=np.int64)
#         offsets[1:] = np.cumsum(np.bincount(labels, minlength=self.n_clusters))

#         dim_per_sub = self.dimension // self.n_subvectors
#         codebooks = np.zeros((self.n_clusters, self.n_subvectors, self.n_clusters_per_subvector, dim_per_sub), np.float32)
#         codewords = np.zeros((n_vectors, self.n_subvectors), dtype=np.uint8)

#         logging.info("Training PQ codebooks...")
#         pos = 0
#         for c in range(self.n_clusters):
#             start, end = int(offsets[c]), int(offsets[c+1])
#             if start == end: continue
#             cluster_vecs = vectors_sorted[pos:pos + end-start]

#             for m in range(self.n_subvectors):
#                 sub = cluster_vecs[:, m*dim_per_sub:(m+1)*dim_per_sub]
#                 pq = MiniBatchKMeans(n_clusters=self.n_clusters_per_subvector,
#                                      random_state=self.random_state, batch_size=1000, max_iter=50)
#                 codes = pq.fit_predict(sub)
#                 codebooks[c, m] = pq.cluster_centers_
#                 codewords[original_ids[pos:pos + end-start], m] = codes
#             pos += end - start

#         # Save as .npy
#         os.makedirs(self.folder_path, exist_ok=True)
#         np.save(os.path.join(self.folder_path, 'centroids.npy'),   cluster_centers)
#         np.save(os.path.join(self.folder_path, 'codebook.npy'),    codebooks)
#         np.save(os.path.join(self.folder_path, 'codewords.npy'),   codewords)
#         np.save(os.path.join(self.folder_path, 'indices.npy'),     original_ids)
#         np.save(os.path.join(self.folder_path, 'offsets.npy'),     offsets)

#         logging.info("Index built and saved as .npy (5 files).")

#     def retreive(self, query_vector, cosine_similarity, get_row, n_clusters=10, top_k=10):
#         folder = self.folder_path

#         # Load only what is needed — everything is small except codewords/indices
#         cluster_centers = np.load(os.path.join(folder, 'centroids.npy'),   mmap_mode='r')           # ~4 KB–40 KB
#         codebook        = np.load(os.path.join(folder, 'codebook.npy'),    mmap_mode='r')           # usually < 200 MB
#         codewords       = np.load(os.path.join(folder, 'codewords.npy'),   mmap_mode='r')           # M × n_subvectors uint8
#         indices         = np.load(os.path.join(folder, 'indices.npy'),     mmap_mode='r')           # M int64
#         offsets         = np.load(os.path.join(folder, 'offsets.npy'),     mmap_mode='r')           # tiny

#         # Coarse search
#         sims = np.array([cosine_similarity(query_vector, c) for c in cluster_centers])
#         probe_clusters = np.argpartition(sims, -n_clusters)[-n_clusters:]

#         cand_ids = []
#         cand_scores = []
#         d = self.dimension // self.n_subvectors

#         for cid in probe_clusters:
#             start, end = int(offsets[cid]), int(offsets[cid+1])
#             if start == end: continue

#             codes = codewords[start:end]           # view, no copy
#             ids   = indices[start:end]             # view, no copy

#             dist = np.zeros(end - start, dtype=np.float32)
#             for m in range(self.n_subvectors):
#                 qm = query_vector[m*d:(m+1)*d]
#                 centers = codebook[cid, m]         # (256, d/m)
#                 dist += np.sum(centers[codes[:, m]]**2, axis=1) - 2 * (centers[codes[:, m]] @ qm)

#             k = min(top_k * 4, len(dist))
#             best = np.argpartition(dist, k-1)[:k]
#             for i in ids[best]:
#                 cand_ids.append(i)
#                 cand_scores.append(cosine_similarity(query_vector, get_row(i)))

#         cand_scores = np.array(cand_scores)
#         top = np.argpartition(-cand_scores, top_k-1)[:top_k]
#         result = np.array(cand_ids)[top]
#         return result[np.argsort(-cand_scores[top])]

In [ ]:
# from typing import Dict, List, Annotated
# import numpy as np
# import os

# DB_SEED_NUMBER = 42
# ELEMENT_SIZE = np.dtype(np.float32).itemsize
# DIMENSION = 64

# class VecDB:
#     def __init__(self, database_file_path = "saved_db.dat", index_file_path = "index", new_db = True, db_size = None) -> None:
#         self.db_path = database_file_path
#         self.index_path = index_file_path
#         self.index=None

#         if self._get_num_records() == 1000000:
#              self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=30,folder_path=self.index_path)
#         elif self._get_num_records() == 10000000:
#              self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=150,folder_path=self.index_path)
#         elif self._get_num_records() == 15000000:
#              self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=250,folder_path=self.index_path)
#         elif self._get_num_records() == 20000000:
#              self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=280,folder_path=self.index_path)
#         if new_db:
#             if db_size is None:
#                 raise ValueError("You need to provide the size of the database")
#             # delete the old DB file if exists
#             if os.path.exists(self.db_path):
#                 os.remove(self.db_path)
#             self.generate_database(db_size)
        
#     def generate_database(self, size: int) -> None:
#         rng = np.random.default_rng(DB_SEED_NUMBER)
#         vectors = rng.random((size, DIMENSION), dtype=np.float32)
#         self._write_vectors_to_file(vectors)
#         # self._build_index()

#     def _write_vectors_to_file(self, vectors: np.ndarray) -> None:
#         mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
#         mmap_vectors[:] = vectors[:]
#         mmap_vectors.flush()

#     def _get_num_records(self) -> int:
#         print(f"Database size (in bytes): {os.path.getsize(self.db_path) // (DIMENSION * ELEMENT_SIZE)}")
#         return os.path.getsize(self.db_path) // (DIMENSION * ELEMENT_SIZE)

#     def insert_records(self, rows: Annotated[np.ndarray, (int, 70)]):
#         num_old_records = self._get_num_records()
#         num_new_records = len(rows)
#         full_shape = (num_old_records + num_new_records, DIMENSION)
#         mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='r+', shape=full_shape)
#         mmap_vectors[num_old_records:] = rows
#         mmap_vectors.flush()
#         #TODO: might change to call insert in the index, if you need
#         self._build_index()

#     def get_one_row(self, row_num: int) -> np.ndarray:
#         # This function is only load one row in memory
#         try:
#             offset = int(row_num) * int(DIMENSION) * int(ELEMENT_SIZE)
#             mmap_vector = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(1, DIMENSION), offset=offset)
#             return np.array(mmap_vector[0])
#         except Exception as e:
#             return f"An error occurred: {e}"

#     def get_all_rows(self) -> np.ndarray:
#         # Take care this load all the data in memory
#         num_records = self._get_num_records()
#         vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
#         return np.array(vectors)
    
#     def retrieve(self, query: Annotated[np.ndarray, (1, DIMENSION)], top_k = 5):
        
#         if self._get_num_records() == 1000000 or self._get_num_records() == 100000:
#             return self.index.retreive(query,self._cal_score,self.get_one_row,n_clusters=30,top_k=top_k)
#         elif self._get_num_records() == 10000000:
#             return self.index.retreive(query,self._cal_score,self.get_one_row,n_clusters=150,top_k=top_k)
#         elif self._get_num_records() == 15000000:
#             return self.index.retreive(query,self._cal_score,self.get_one_row,n_clusters=250,top_k=top_k)
#         elif self._get_num_records() == 20000000:
#             return self.index.retreive(query,self._cal_score,self.get_one_row,n_clusters=280,top_k=top_k)
    
#     def _cal_score(self, vec1, vec2):
#         dot_product = np.dot(vec1, vec2)
#         norm_vec1 = np.linalg.norm(vec1)    
#         norm_vec2 = np.linalg.norm(vec2)
#         cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
#         return cosine_similarity

#     def _build_index(self):
#         # Placeholder for index building logic

#         if self._get_num_records() == 1000000 or self._get_num_records() == 100000:
#              print("Building index for 1M or 100K records")
#              self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=30,folder_path=self.index_path)
#         elif self._get_num_records() == 10000000:
#              print("Building index for 10M records")
#              self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=150,folder_path=self.index_path)
#         elif self._get_num_records() == 15000000:
#              print("Building index for 15M records")
#              self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=250,folder_path=self.index_path)
#         elif self._get_num_records() == 20000000:
#             print("Building index for 20M records")
#             self.index = IVF_PQ(n_subvectors=8,n_bits=8,n_clusters=280,folder_path=self.index_path)
#         self.index.build(self.get_all_rows())
      



In [ ]:
# import json
# import numpy as np
# import os
# import shutil
# import tqdm
# import heapq
# from sklearn.cluster import MiniBatchKMeans
# from typing import Annotated
# import varint

# DB_SEED_NUMBER = 42
# ELEMENT_SIZE = np.dtype(np.float32).itemsize
# DIMENSION = 64

# class VecDB:
#     def __init__(self, database_file_path="saved_db.dat", index_file_path="index.dat", new_db=True, db_size=None) -> None:
#         self.db_path = database_file_path
#         self.index_path = index_file_path
#         self.dim = db_size
#         if new_db:
#             if db_size is None:
#                 raise ValueError("You need to provide the size of the database")
#             if os.path.exists(self.db_path):
#                 os.remove(self.db_path)
#             self.generate_database(db_size)

#     def generate_database(self, size: int) -> None:
#         vectors = np.memmap("new_embeddings.dat", dtype=np.float32, mode='r', shape=(size, DIMENSION))
#         self._write_vectors_to_file(vectors)
#         self._build_index()

#     def _write_vectors_to_file(self, vectors: np.ndarray) -> None:
#         mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
#         mmap_vectors[:] = vectors[:]
#         mmap_vectors.flush()

#     def _get_num_records(self) -> int:
#         return os.path.getsize(self.db_path) // (DIMENSION * ELEMENT_SIZE)

#     def insert_records(self, rows: Annotated[np.ndarray, (int, 64)]):
#         num_old_records = self._get_num_records()
#         num_new_records = len(rows)
#         full_shape = (num_old_records + num_new_records, DIMENSION)
#         mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='r+', shape=full_shape)
#         mmap_vectors[num_old_records:] = rows
#         mmap_vectors.flush()
#         self._build_index()

#     def get_one_row(self, row_num: int) -> np.ndarray:
#         try:
#             offset = row_num * DIMENSION * ELEMENT_SIZE
#             mmap_vector = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(1, DIMENSION), offset=offset)
#             return np.array(mmap_vector[0])
#         except Exception as e:
#             raise

#     def get_all_rows(self) -> np.ndarray:
#         num_records = self._get_num_records()
#         vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
#         return np.array(vectors)

#     def get_all_ids_rows(self, ids) -> np.ndarray:
#         num_records = self._get_num_records()
#         vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
#         return vectors[ids]

#     def unpack_packed_codes(self, packed_block):
#         # packed_block: uint8 array shape (n, packed_cols)
#         # output: uint8 array shape (n, M)
#         # high nibble = code for 2*i (>>4), low nibble = code for 2*i+1 (&0x0F)
#         M = 8  # number of sub-vectors (hard-coded to your m)
#         if packed_block.ndim == 1:
#             packed_block = packed_block.reshape(1, -1)
#         n = packed_block.shape[0]
#         codes = np.empty((n, M), dtype=np.uint8)
#         # vectorized unpack
#         high = (packed_block >> 4).astype(np.uint8)
#         low  = (packed_block & 0x0F).astype(np.uint8)
#         codes[:, 0::2] = high
#         codes[:, 1::2] = low
#         return codes

#     ############################################################################
#     # -------------------------- INDEX BUILDING ------------------------------ #
#     ############################################################################
#     def _build_index(self, n_coarse=256, m=8, bits=8, batch_size=5000):
#         N, D = self._get_num_records(), DIMENSION
#         assert D % m == 0
#         sub_dim = D // m
#         k_sub = 1 << bits

#         data = self.get_all_rows()
#         print(f"Building IVF-PQ: N={N}, m={m}, k={k_sub}, nlist={n_coarse}")

#         # clean index folder
#         if os.path.exists(self.index_path):
#             shutil.rmtree(self.index_path)
#         os.makedirs(self.index_path, exist_ok=True)

#         # -------------------------------------------------------
#         # 1) Train coarse KMeans
#         # -------------------------------------------------------
#         coarse_km = MiniBatchKMeans(
#             n_clusters=n_coarse,
#             batch_size=10_000,
#             random_state=DB_SEED_NUMBER
#         )
#         coarse_labels = coarse_km.fit_predict(data)
#         coarse_centroids = coarse_km.cluster_centers_.astype(np.float32)
#         np.save(os.path.join(self.index_path, "coarse_centroids.npy"), coarse_centroids)

#         # -------------------------------------------------------
#         # 2) Train PQ codebooks
#         # -------------------------------------------------------
#         residuals = data - coarse_centroids[coarse_labels]
#         codebook = np.zeros((m, k_sub, sub_dim), dtype=np.float32)

#         for i in range(m):
#             sub = residuals[:, i*sub_dim:(i+1)*sub_dim]
#             km = MiniBatchKMeans(
#                 n_clusters=k_sub,
#                 batch_size=20_000,
#                 random_state=DB_SEED_NUMBER
#             )
#             km.fit(sub)
#             codebook[i] = km.cluster_centers_.astype(np.float32)

#         np.save(os.path.join(self.index_path, "pq_codebook.npy"), codebook)

#         # -------------------------------------------------------
#         # 3) Encode PQ codes
#         # -------------------------------------------------------
#         codes = np.zeros((N, m), dtype=np.uint8)

#         for i in range(m):
#             subvecs = residuals[:, i*sub_dim:(i+1)*sub_dim]
#             cents = codebook[i]
#             cents_sq = np.sum(cents*cents, axis=1)

#             for start in range(0, N, batch_size):
#                 end = min(start + batch_size, N)
#                 batch = subvecs[start:end]
#                 batch_sq = np.sum(batch*batch, axis=1, keepdims=True)
#                 cross = batch @ cents.T
#                 dists = batch_sq + cents_sq[None,:] - 2*cross
#                 codes[start:end, i] = np.argmin(dists, axis=1)

#         # -------------------------------------------------------
#         # 4) Build inverted lists
#         # -------------------------------------------------------
#         lists = [[] for _ in range(n_coarse)]
#         for idx, c in enumerate(coarse_labels):
#             lists[c].append(idx)

#         offsets = np.zeros(n_coarse + 1, dtype=np.uint64)

#         ivf_ids_file = os.path.join(self.index_path, "ivf_ids.bin")
#         ivf_codes_file = os.path.join(self.index_path, "ivf_codes.bin")

#         with open(ivf_ids_file, "wb") as f_ids, open(ivf_codes_file, "wb") as f_codes:
#             count = 0
#             for c in range(n_coarse):
#                 ids = np.array(lists[c], dtype=np.uint32)
#                 n_vec = ids.size

#                 # prefix sum offset
#                 offsets[c+1] = offsets[c] + n_vec

#                 if n_vec == 0:
#                     continue

#                 # write raw IDs (no delta, no varint)
#                 ids.tofile(f_ids)

#                 # write raw PQ codes aligned with IDs
#                 f_codes.write(codes[ids].astype(np.uint8).tobytes())

#         offsets.tofile(os.path.join(self.index_path, "ivfpq_offsets.bin"))

#         print("IVF-PQ index built (raw IDs + raw PQ codes).")

#     def retrieve(self, query, top_k=10, nprobe=64, candidate_multiplier=8):
#         q = np.asarray(query, dtype=np.float32)
#         m = 8
#         sub_dim = DIMENSION // m
#         k = 256

#         index = self.index_path
#         coarse_centroids = np.load(os.path.join(index, "coarse_centroids.npy"))
#         codebook = np.load(os.path.join(index, "pq_codebook.npy"))
#         offsets = np.fromfile(os.path.join(index, "ivfpq_offsets.bin"), dtype=np.uint64)

#         nlist = coarse_centroids.shape[0]

#         # ----------------------------- Probe ------------------------------
#         diff = coarse_centroids - q
#         cdists = np.sum(diff*diff, axis=1)

#         top_list = np.argsort(cdists)[:min(nprobe, nlist)]

#         # distance tables for each list later
#         ids_file = open(os.path.join(index, "ivf_ids.bin"), "rb")
#         codes_file = open(os.path.join(index, "ivf_codes.bin"), "rb")

#         # candidate heap
#         candidate_heap = []
#         n_candidates = top_k * candidate_multiplier

#         for c in top_list:
#             start = int(offsets[c])
#             end = int(offsets[c+1])
#             n_vec = end - start
#             if n_vec == 0:
#                 continue

#             # ----------------------------- Read IDs ------------------------------
#             ids_file.seek(start * 4)
#             ids = np.frombuffer(ids_file.read(n_vec*4), dtype=np.uint32)

#             # --------------------------- Read PQ codes (raw) ----------------------
#             codes_file.seek(start * m)
#             raw_codes = np.frombuffer(codes_file.read(n_vec*m), dtype=np.uint8)
#             raw_codes = raw_codes.reshape(n_vec, m)

#             # --------------------------- Build Distance Table ---------------------
#             centroid_c = coarse_centroids[c]
#             q_res = q - centroid_c
#             sub_q = q_res.reshape(m, sub_dim)

#             diff = codebook - sub_q[:, None, :]
#             dist_table = np.sum(diff*diff, axis=2)

#             # ---------------------- Compute PQ distances --------------------------
#             # fast lookup
#             pq_dists = dist_table[np.arange(m)[:,None], raw_codes.T].sum(axis=0)

#             # ---------------------- Push candidates -------------------------------
#             for vid, dist in zip(ids, pq_dists):
#                 if len(candidate_heap) < n_candidates:
#                     heapq.heappush(candidate_heap, (-dist, int(vid)))
#                 else:
#                     if -dist > candidate_heap[0][0]:
#                         heapq.heapreplace(candidate_heap, (-dist, int(vid)))

#         ids_file.close()
#         codes_file.close()

#         # ------------------------- Exact rerank -------------------------------
#         if not candidate_heap:
#             return []

#         candidates = {vid for _, vid in candidate_heap}

#         result_heap = []
#         q_normalized = q / np.linalg.norm(q)
#         for vid in candidates:
#             vec = self.get_one_row(int(vid))
           
#             score = np.dot(q_normalized,  vec)

#             if len(result_heap) < top_k:
#                 heapq.heappush(result_heap, (score, int(vid)))
#             else:
#                 if score > result_heap[0][0]:
#                     heapq.heapreplace(result_heap, (score, int(vid)))

#         result_heap.sort(reverse=True)

#         result = [vid for score, vid in result_heap]
#         print(result)
#         return result



In [111]:
import json
import numpy as np
import os
import shutil
import tqdm
import heapq
from sklearn.cluster import MiniBatchKMeans
from typing import Annotated

DB_SEED_NUMBER = 42
ELEMENT_SIZE = np.dtype(np.float32).itemsize
DIMENSION = 64

class VecDB:
    def __init__(self, database_file_path="saved_db.dat", index_file_path="index.dat", new_db=True, db_size=None) -> None:
        self.db_path = database_file_path
        self.index_path = index_file_path
        self.dim = db_size
        if new_db:
            if db_size is None:
                raise ValueError("You need to provide the size of the database")
            if os.path.exists(self.db_path):
                os.remove(self.db_path)
            self.generate_database(db_size)

    def generate_database(self, size: int) -> None:
        vectors = np.memmap("new_embeddings.dat", dtype=np.float32, mode='r', shape=(size, DIMENSION))
        self._write_vectors_to_file(vectors)
        self._build_index()

    def _write_vectors_to_file(self, vectors: np.ndarray) -> None:
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
        mmap_vectors[:] = vectors[:]
        mmap_vectors.flush()

    def _get_num_records(self) -> int:
        return os.path.getsize(self.db_path) // (DIMENSION * ELEMENT_SIZE)

    def insert_records(self, rows: Annotated[np.ndarray, (int, 64)]):
        num_old_records = self._get_num_records()
        num_new_records = len(rows)
        full_shape = (num_old_records + num_new_records, DIMENSION)
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='r+', shape=full_shape)
        mmap_vectors[num_old_records:] = rows
        mmap_vectors.flush()
        self._build_index()

    def get_one_row(self, row_num: int) -> np.ndarray:
        try:
            offset = row_num * DIMENSION * ELEMENT_SIZE
            mmap_vector = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(1, DIMENSION), offset=offset)
            return np.array(mmap_vector[0])
        except Exception as e:
            raise

    def get_all_rows(self) -> np.ndarray:
        num_records = self._get_num_records()
        vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
        return np.array(vectors)

    def get_all_ids_rows(self, ids) -> np.ndarray:
        num_records = self._get_num_records()
        vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
        return vectors[ids]

    ############################################################################
    # -------------------------- Retrieval (search) --------------------------- #
    ############################################################################
    def retrieve(self, query, top_k=10, nprobe=40, candidate_multiplier=8):
        """
        IVF-PQ search:
          - find top-nprobe coarse cells (by L2)
          - for each cell, read list of (ids, pq-codes)
          - compute approximate PQ distances using the *query residual* (q - coarse_centroid)
          - keep top candidates, then re-rank by exact dot or L2 (exact)
        """
        q = np.asarray(query, dtype=np.float32).ravel()
        q_norm = np.linalg.norm(q)
        if q_norm > 1e-12:
            q = q / q_norm  # optional normalization if your database vectors are normalized; keep consistent with indexing

        # constants (should match build)
        D = DIMENSION
        # load index metadata
        index_path = self.index_path
        coarse_centroids = np.load(os.path.join(index_path, "coarse_centroids.npy"))  # shape (n_coarse, D)
       
        n_coarse =256
        m = 8
        sub_dim = 8
        k_sub = 256

        # load codebook (m, k_sub, sub_dim)
        codebook = np.load(os.path.join(index_path, "pq_codebook.npy"))
        # offsets for inverted lists (uint64, length n_coarse+1), offsets in number-of-vectors units per list
        offsets = np.fromfile(os.path.join(index_path, "ivfpq_offsets.bin"), dtype=np.uint64)
        ivfpq_path = os.path.join(index_path, "ivfpq.bin")

        # 1) find top-nprobe coarse cells by L2 distance between q and coarse_centroid
        # compute squared L2 distances
        diffs = coarse_centroids - q  # shape (n_coarse, D)
        coarse_dists = np.sum(diffs * diffs, axis=1)
        top_coarse_idx = np.argpartition(coarse_dists, nprobe - 1)[:nprobe]
        top_coarse_idx = top_coarse_idx[np.argsort(coarse_dists[top_coarse_idx])]  # from closest to far
        # We'll need the coarse centroids for each chosen cell
        # Precompute query residual per chosen centroid when needed.

        n_candidates = top_k * candidate_multiplier
        candidate_heap = []  # min-heap of (approx_dist, vid) where smaller = better (L2)

        # compute how many bytes the codebook takes (written at file start)
       

        # For each coarse cell, read its list and compute approximate distances
        with open(ivfpq_path, "rb") as f:
            for c in top_coarse_idx:
                start = int(offsets[c])
                end = int(offsets[c + 1])
                n_vec = end - start
                if n_vec == 0:
                    continue

                # compute residual query: q - centroid_c
                centroid_c = coarse_centroids[c]
                query_residual = q - centroid_c
                sub_q = query_residual.reshape(m, sub_dim)  # shape (m, sub_dim)

                # compute dist_table for this centroid: for each subvector slot, a vector of size k_sub: || sub_q - codebook[i,k] ||^2
                # codebook shape (m, k_sub, sub_dim)
                # dist_table shape (m, k_sub)
                diff = codebook - sub_q[:, np.newaxis, :]  # broadcasting: (m,k_sub,sub_dim) - (m,1,sub_dim)
                dist_table = np.sum(diff * diff, axis=2)  # (m, k_sub)
                del diff

                # position in file to read this list:
                # file layout: [codebook bytes] [for each cell: ids(uint32) then codes(uint8 flattened)] contiguously
                # We stored each cell sequentially in that order during build.
                # We need to know the byte start for this cell. During building we wrote lists in order and tracked offsets in "vector counts".
                # To compute the byte offset we need to walk previous lists lengths; but we stored offsets in vector-count units.
                # We'll compute byte-position as: codebook_nbytes + sum_{i<cell}(len(list_i) * (4 + m))
                # That can be computed by reading offsets array (which stores cumulative counts) => offsets[c] gives number of vectors before cell c.
                byte_pos =  int(offsets[c]) * (4 + m)  # each vector stored as uint32 id (4) + m bytes codes (uint8)
                f.seek(byte_pos)
                block_bytes = f.read(n_vec * (4 + m))

                # parse ids and codes
                ids = np.frombuffer(block_bytes, dtype=np.uint32, count=n_vec, offset=0)
                codes = np.frombuffer(block_bytes, dtype=np.uint8, offset=n_vec * 4).reshape(n_vec, m)
                # now compute approx distances per vector using dist_table
                # dist_table: (m, k_sub)
                # codes.T shape: (m, n_vec)
                # pick dist_table[range(m), codes[i]] for each i
                # Efficient: dist_table[np.arange(m)[:,None], codes.T] -> shape (m, n_vec) -> sum over m -> (n_vec,)
                picked = dist_table[np.arange(m)[:, None], codes.T]  # (m, n_vec)
                approx_dists = picked.sum(axis=0)  # (n_vec,)
                del picked, dist_table, block_bytes

                # push candidates into a min-heap (we want smallest L2 distances)
                for dist, vid in zip(approx_dists, ids):
                    if len(candidate_heap) < n_candidates:
                        heapq.heappush(candidate_heap, (-dist, int(vid)))  # use -dist to keep largest negative = smallest dist if we want max-heap semantics
                    else:
                        # candidate_heap holds negative-dist; smallest dist corresponds to largest -dist
                        if -dist > candidate_heap[0][0]:
                            heapq.heapreplace(candidate_heap, (-dist, int(vid)))

        if not candidate_heap:
            return []

        # convert candidate_heap to list of unique vids (we might deduplicate)
        cand = {vid for _, vid in candidate_heap}
        # now exact re-ranking: load each candidate vector and compute exact similarity or exact L2
        # We'll compute dot(q, vec) if vectors and q are normalized; otherwise compute negative L2 for ranking.
        result_heap = []  # min-heap of (score, vid) where larger score is better

        # Decide exact metric: if your DB vectors are normalized, use dot product; else L2 (smaller is better)
        # We'll use dot product assuming normalization above. Keep consistent with indexing/build normalization.
        for vid in cand:
            vec = self.get_one_row(int(vid)).astype(np.float32)
            # If your vectors were normalized at index time, use dot; otherwise compute -L2 to get descending order
            exact_score = float(np.dot(vec, q))
            if len(result_heap) < top_k:
                heapq.heappush(result_heap, (exact_score, int(vid)))
            else:
                if exact_score > result_heap[0][0]:
                    heapq.heapreplace(result_heap, (exact_score, int(vid)))

        # sort results descending by score
        result_heap.sort(reverse=True)
        final_ids = [int(vid) for score, vid in result_heap]

        return final_ids

    ############################################################################
    # ---------------------------- Index building ---------------------------- #
    ############################################################################
    def _build_index(self, n_coarse=256, m=8, bits=8, batch_size=5000):
        """
        Build IVF-PQ index memory-safely.
        - n_coarse: number of coarse clusters
        - m: number of PQ sub-vectors
        - bits: number of bits per sub-vector (k_sub = 2^bits)
        - batch_size: used for PQ encoding batching
        """
        N, D = self._get_num_records(), DIMENSION
        assert D % m == 0, f"D={D} must be divisible by m={m}"
        sub_dim = D // m
        k_sub = 1 << bits

        data = self.get_all_rows()  # careful: loads all to memory; for huge DB you'd batch-training differently

        # remove old index
        if os.path.exists(self.index_path):
            shutil.rmtree(self.index_path)
        os.makedirs(self.index_path, exist_ok=True)

        # === 1. Coarse quantizer (L2) ===
        print("Training coarse quantizer...")

        coarse_km = MiniBatchKMeans(n_clusters=n_coarse, batch_size=10_000, random_state=DB_SEED_NUMBER)
        coarse_labels = coarse_km.fit_predict(data)
        coarse_cents = coarse_km.cluster_centers_.astype(np.float32)  # shape (n_coarse, D)


        #normalize centroid 
        coarse_cents = coarse_cents / (np.linalg.norm(coarse_cents, axis=1, keepdims=True) + 1e-12)

        np.save(os.path.join(self.index_path, "coarse_centroids.npy"), coarse_cents)

        # === 2. Compute residuals and train PQ codebooks on residuals ===
        print("Training PQ codebooks on residuals...")
        # residual for each vector: x - centroid_of_cell

        residuals = data - coarse_cents[coarse_labels]
        residuals = residuals / (np.linalg.norm(residuals, axis=1, keepdims=True) + 1e-12)
        
        codebook = np.zeros((m, k_sub, sub_dim), dtype=np.float32)

        for i in range(m):
            subvecs = residuals[:, i * sub_dim:(i + 1) * sub_dim]
            km = MiniBatchKMeans(n_clusters=k_sub, batch_size=20_000, random_state=DB_SEED_NUMBER)
            #normalize subvecs
           
            km.fit(subvecs)
            codebook[i] = km.cluster_centers_.astype(np.float32)

        
        np.save(os.path.join(self.index_path, "pq_codebook.npy"), codebook)
       
        # === 3. Encode all vectors using PQ (batching to avoid huge memory) ===
        print("Encoding vectors into PQ codes (memory-safe)...")
        codes = np.zeros((N, m), dtype=np.uint8)  # MUST be uint8 to store codes 0..k_sub-1
        for i in range(m):
            subvecs = residuals[:, i * sub_dim:(i + 1) * sub_dim]  # residual sub-vector
            cents = codebook[i]  # shape (k_sub, sub_dim)
            cents_sq = np.sum(cents * cents, axis=1)  # (k_sub,)

            for start in range(0, N, batch_size):
                end = min(start + batch_size, N)
                batch = subvecs[start:end]  # (B, sub_dim)
                batch_sq = np.sum(batch * batch, axis=1, keepdims=True)  # (B,1)
                cross = batch @ cents.T  # (B, k_sub)
                dists = batch_sq + cents_sq[None, :] - 2.0 * cross  # (B, k_sub)
                codes[start:end, i] = np.argmin(dists, axis=1).astype(np.uint8)

        # === 4. Build inverted lists (IVF) ===
        print("Building inverted lists...")
        lists = [[] for _ in range(n_coarse)]
        for idx, c in enumerate(coarse_labels):
            lists[c].append(idx)

        # === 5. Write index to disk: layout = [codebook bytes][for each cell: ids(uint32) then codes(uint8*m)] ===
        print("Writing final index...")
        offsets = np.zeros(n_coarse + 1, dtype=np.uint64)
        ivfpq_file = os.path.join(self.index_path, "ivfpq.bin")
        with open(ivfpq_file, "wb") as f:
           

            # write each cell's ids then codes
            total_written = 0
            for c in range(n_coarse):
                ids = np.array(lists[c], dtype=np.uint32)
                n_vec = ids.size
                offsets[c + 1] = offsets[c] + n_vec
                if n_vec == 0:
                    continue
                subset_codes = codes[ids].astype(np.uint8)  # shape (n_vec, m)
                ids.tofile(f)  # uint32 sequence
                subset_codes.tofile(f)  # raw uint8 bytes (n_vec * m)
                total_written += n_vec

        offsets.tofile(os.path.join(self.index_path, "ivfpq_offsets.bin"))
        # Save meta already saved
        print(f"IVF-PQ built! Index file: {ivfpq_file}, size: {os.path.getsize(ivfpq_file) / 1e6:.1f} MB")
#ivf pq better 


# 10M	score	-192.33333333333334	time	0.62	RAM	1.29 MB


# 1M	score	-108.66666666666667	time	0.07	RAM	0.23 MB
# 10M	score	-192.33333333333334	time	0.65	RAM	1.39 MB
# 15M	score	-174.0	time	1.21	RAM	3.21 MB






# 1M	score	-108.66666666666667	time	0.23	RAM	0.23 MB   before normailzation distance

# 1M	score	-108.66666666666667	time	0.07	RAM	0.16 MB    after 



In [5]:
import numpy as np
import os

DIMENSION = 64
def create_other_DB_size(input_file, output_file, target_rows, embedding_dim = DIMENSION):
    # Configuration
    dtype = 'float32'

    # 1. Determine the shape of the source file
    # We calculate rows based on file size to be safe, or you can hardcode 20_000_000
    file_size_bytes = os.path.getsize(input_file)
    itemsize = np.dtype(dtype).itemsize
    total_rows = file_size_bytes // (embedding_dim * itemsize)

    print(f"Source detected: {total_rows} rows.")

    # 2. Open source in read mode ('r')
    # This uses almost 0 RAM, it just points to the file on disk
    source_memmap = np.memmap(
        input_file,
        dtype=dtype,
        mode='r',
        shape=(total_rows, embedding_dim)
    )

    # 3. Create the new file in write mode ('w+')
    # We define the shape as the target size (1M, 64)
    dest_memmap = np.memmap(
        output_file,
        dtype=dtype,
        mode='w+',
        shape=(target_rows, embedding_dim)
    )

    # 4. Copy the data
    # This transfers the binary blocks directly
    print("Copying data...")
    dest_memmap[:] = source_memmap[:target_rows]

    # 5. Flush to save changes to disk
    dest_memmap.flush()

    print(f"Success! Saved first {target_rows} rows to {output_file}")

In [6]:
PATH_DB_VECTORS_20M = "OpenSubtitles_en_20M_emb_64.dat"
PATH_DB_VECTORS_10M = "OpenSubtitles_en_10M_emb_64.dat"
PATH_DB_VECTORS_1M = "OpenSubtitles_en_1M_emb_64.dat"

In [7]:
if not os.path.exists(PATH_DB_VECTORS_1M):
    create_other_DB_size(PATH_DB_VECTORS_20M, PATH_DB_VECTORS_1M, 1_000_000)
if not os.path.exists(PATH_DB_VECTORS_10M):
    create_other_DB_size(PATH_DB_VECTORS_20M, PATH_DB_VECTORS_10M, 10_000_000)

These are the functions for running and reporting

In [8]:
import numpy as np
import os
import time
from dataclasses import dataclass
from typing import List
from memory_profiler import memory_usage
import gc

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]

def run_queries(db, queries, top_k, actual_ids, num_runs):
    """
    Run queries on the database and record results for each query.

    Parameters:
    - db: Database instance to run queries on.
    - queries: List of query vectors.
    - top_k: Number of top results to retrieve.
    - actual_ids: List of actual results to evaluate accuracy.
    - num_runs: Number of query executions to perform for testing.

    Returns:
    - List of Result
    """
    global results
    results = []
    for i in range(num_runs):
        tic = time.time()
        db_ids = db.retrieve(queries[i], top_k)
        toc = time.time()
        run_time = toc - tic
        results.append(Result(run_time, top_k, db_ids, actual_ids[i]))
    return results

def memory_usage_run_queries(args):
    """
    Run queries and measure memory usage during the execution.

    Parameters:
    - args: Arguments to be passed to the run_queries function.

    Returns:
    - results: The results of the run_queries.
    - memory_diff: The difference in memory usage before and after running the queries.
    """
    global results
    mem_before = max(memory_usage())
    mem = memory_usage(proc=(run_queries, args, {}), interval = 1e-3)
    return results, max(mem) - mem_before

def evaluate_result(results: List[Result]):
    """
    Evaluate the results based on accuracy and runtime.
    Scores are negative. So getting 0 is the best score.

    Parameters:
    - results: A list of Result objects

    Returns:
    - avg_score: The average score across all queries.
    - avg_runtime: The average runtime for all queries.
    """
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append( -1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
            except:
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)

def get_actual_ids_first_k(actual_sorted_ids, k):
    """
    Retrieve the IDs from the sorted list of actual IDs.
    actual IDs has the top_k for the 20 M database but for other databases we have to remove the numbers higher than the max size of the DB.

    Parameters:
    - actual_sorted_ids: A list of lists containing the sorted actual IDs for each query.
    - k: The DB size.

    Returns:
    - List of lists containing the actual IDs for each query for this DB.
    """
    return [[id for id in actual_sorted_ids_one_q if id < k] for actual_sorted_ids_one_q in actual_sorted_ids]

This code to actually run the class you have been implemented. The `VecDB` class should take the database path, and index path that you provided.<br>
Note at the submission I'll not run the insert records. <br>
The query istelf will be changed at submissions day but not the DB

In [14]:
# check memory usage for the import line independently
import tracemalloc
tracemalloc.start()
start_snapshot = tracemalloc.take_snapshot()

# from vec_db import VecDB

end_snapshot = tracemalloc.take_snapshot()
stats = end_snapshot.compare_to(start_snapshot, 'lineno')
for stat in stats[:5]:  # show top differences
    print(stat)

tracemalloc.stop()

d:\Apps\anaconda\Lib\tracemalloc.py:560: size=296 B (+296 B), count=2 (+2), average=148 B
d:\Apps\anaconda\Lib\tracemalloc.py:423: size=296 B (+296 B), count=2 (+2), average=148 B
C:\Users\m\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3517: size=296 B (+0 B), count=1 (+0), average=296 B
d:\Apps\anaconda\Lib\codeop.py:126: size=286 B (+0 B), count=2 (+0), average=143 B
C:\Users\m\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3577: size=160 B (+0 B), count=1 (+0), average=160 B


In [10]:
results = []
to_print_arr = []

In [108]:
print("Team Number", 3)
database_info = {
    "1M": {
        "database_file_path": PATH_DB_VECTORS_1M,
        "index_file_path": "index_1M_edit"
        "",
        "size": 10**6
    },
    # "10M": {
    #     "database_file_path": PATH_DB_VECTORS_10M,
    #     "index_file_path": "index_10M_edit",
    #     "size": 10 * 10**6
    # },
    # "20M": {
    #     "database_file_path": PATH_DB_VECTORS_1M,
    #     "index_file_path": "index_20M_edit",
    #     "size": 20 * 10**6
    # }
}

Team Number 3


In [109]:
gc.collect()


782

In [112]:
for db_name, info in database_info.items():
    db = VecDB(database_file_path = info["database_file_path"], index_file_path = info["index_file_path"], new_db = False)
    db._build_index()

Training coarse quantizer...
Training PQ codebooks on residuals...
Encoding vectors into PQ codes (memory-safe)...
Building inverted lists...
Writing final index...
IVF-PQ built! Index file: index_1M_edit\ivfpq.bin, size: 12.0 MB
